# 如何删除消息

图的常见状态之一是消息列表。通常您只向该状态添加消息。但是，有时您可能想要删除消息（通过直接修改状态或作为图形的一部分）。为此，您可以使用 `RemoveMessage` 修饰符。在本指南中，我们将介绍如何做到这一点。

关键思想是每个状态键都有一个 `reducer` 键。该键指定如何组合状态更新。预构建的 [`MessagesAnnotation`](/langgraphjs/concepts/low_level/#messagesannotation) 有一个 messages 键，并且该键的减速器接受这些 `RemoveMessage` 修饰符。然后该减速器使用这些 `RemoveMessage` 从键中删除消息。

因此请注意，仅仅因为您的图形状态有一个作为消息列表的键，并不意味着此 `RemoveMessage` 修饰符将起作用。您还必须定义一个知道如何使用它的 `reducer`。

**注意**：许多模型期望有关消息列表的某些规则。例如，有些人希望它们以 `user` 消息开头，其他人则希望所有带有工具调用的消息后面都跟有工具消息。**删除消息时，您需要确保不违反这些规则。**

## 设置

首先，安装本示例所需的依赖项：
```bash
npm install @langchain/langgraph @langchain/openai @langchain/core zod uuid
```
接下来，我们需要为 OpenAI 设置 API 密钥（我们将使用的 LLM）：
```typescript
process.env.OPENAI_API_KEY = 'YOUR_API_KEY';
```
或者，我们可以为 [LangSmith 追踪](https://smith.langchain.com/) 设置 API 密钥，这将为我们提供一流的可观察性。
```typescript
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_API_KEY = "YOUR_API_KEY";
```
现在，让我们构建一个使用消息的简单图表。

## 构建代理
现在让我们构建一个简单的 ReAct 风格代理。

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from "@langchain/core/tools";
import { MemorySaver } from "@langchain/langgraph-checkpoint";
import { MessagesAnnotation, StateGraph, START, END } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { z } from "zod";

const memory = new MemorySaver();

const search = tool((_) => {
  // 这是实际实现的占位符
  // 不过不要让LLM知道这一点😊
  return [
    "It's sunny in San Francisco, but you better look out if you're a Gemini 😈.",
  ];
}, {
  name: "search",
  description: "Call to surf the web.",
  schema: z.object({
    query: z.string(),
  })
});

const tools = [search];
const toolNode = new ToolNode<typeof MessagesAnnotation.State>(tools);
const model = new ChatOpenAI({ model: "gpt-4o" });
const boundModel = model.bindTools(tools);

function shouldContinue(state: typeof MessagesAnnotation.State): "action" | typeof END {
  const lastMessage = state.messages[state.messages.length - 1];
  if (
    "tool_calls" in lastMessage &&
    Array.isArray(lastMessage.tool_calls) &&
    lastMessage.tool_calls.length
  ) {
    return "action";
  }
  // 如果没有工具调用，那么我们就完成了
  return END;
}

// 定义调用模型的函数
async function callModel(state: typeof MessagesAnnotation.State) {
  const response = await boundModel.invoke(state.messages);
  return { messages: [response] };
}

// 定义一个新图
const workflow = new StateGraph(MessagesAnnotation)
  // 定义我们将在其之间循环的两个节点
  .addNode("agent", callModel)
  .addNode("action", toolNode)
  // 将入口点设置为“agent”
  // 这意味着该节点是第一个被调用的节点
  .addEdge(START, "agent")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“代理”。
    // 这意味着这些是调用“agent”节点后获取的边。
    "agent",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue
  )
  // 我们现在添加从“tools”到“agent”的正常边缘。
  // 这意味着在调用“tools”之后，接下来调用“agent”节点。
  .addEdge("action", "agent");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const app = workflow.compile({ checkpointer: memory });

In [2]:
import { HumanMessage } from "@langchain/core/messages";
import { v4 as uuidv4 } from "uuid";

const config = { configurable: { thread_id: "2" }, streamMode: "values" as const };
const inputMessage = new HumanMessage({
  id: uuidv4(),
  content: "hi! I'm bob",
});

for await (const event of await app.stream(
  { messages: [inputMessage] },
  config,
)) {
  const lastMsg = event.messages[event.messages.length - 1];
  console.dir(
    {
      type: lastMsg._getType(),
      content: lastMsg.content,
      tool_calls: lastMsg.tool_calls,
    },
    { depth: null }
  )
}

const inputMessage2 = new HumanMessage({
  id: uuidv4(),
  content: "What's my name?",
});
for await (const event of await app.stream(
  { messages: [inputMessage2] },
  config,
)) {
  const lastMsg = event.messages[event.messages.length - 1];
  console.dir(
    {
      type: lastMsg._getType(),
      content: lastMsg.content,
      tool_calls: lastMsg.tool_calls,
    },
    { depth: null }
  )
}

{ type: 'human', content: "hi! I'm bob", tool_calls: undefined }
{
  type: 'ai',
  content: 'Hi Bob! How can I assist you today?',
  tool_calls: []
}
{ type: 'human', content: "What's my name?", tool_calls: undefined }
{ type: 'ai', content: 'Your name is Bob.', tool_calls: [] }


## 手动删除消息

首先，我们将介绍如何手动删除消息。我们看一下线程当前的状态：

In [3]:
const messages = (await app.getState(config)).values.messages;
console.dir(
  messages.map((msg) => ({
    id: msg.id,
    type: msg._getType(),
    content: msg.content,
    tool_calls:
    msg.tool_calls,
  })),
  { depth: null }
);

[
  {
    id: '24187daa-00dd-40d8-bc30-f4e24ff78165',
    type: 'human',
    content: "hi! I'm bob",
    tool_calls: undefined
  },
  {
    id: 'chatcmpl-9zYV9yHLiZmR2ZVHEhHcbVEshr3qG',
    type: 'ai',
    content: 'Hi Bob! How can I assist you today?',
    tool_calls: []
  },
  {
    id: 'a67e53c3-5dcf-4ddc-83f5-309b72ac61f4',
    type: 'human',
    content: "What's my name?",
    tool_calls: undefined
  },
  {
    id: 'chatcmpl-9zYV9mmpJrm3SQ7ngMJZ1XBHzHfL6',
    type: 'ai',
    content: 'Your name is Bob.',
    tool_calls: []
  }
]


我们可以调用 `updateState` 并传入第一条消息的 id。这将删除该消息。

In [4]:
import { RemoveMessage } from "@langchain/core/messages";

await app.updateState(config, { messages: new RemoveMessage({ id: messages[0].id }) })

{
  configurable: {
    thread_id: '2',
    checkpoint_ns: '',
    checkpoint_id: '1ef61abf-1fc2-6431-8005-92730e9d667c'
  }
}


如果我们现在查看这些消息，我们可以验证第一条消息是否已删除。

In [5]:
const updatedMessages = (await app.getState(config)).values.messages;
console.dir(
  updatedMessages.map((msg) => ({
    id: msg.id,
    type: msg._getType(),
    content: msg.content,
    tool_calls:
    msg.tool_calls,
  })),
  { depth: null }
);

[
  {
    id: 'chatcmpl-9zYV9yHLiZmR2ZVHEhHcbVEshr3qG',
    type: 'ai',
    content: 'Hi Bob! How can I assist you today?',
    tool_calls: []
  },
  {
    id: 'a67e53c3-5dcf-4ddc-83f5-309b72ac61f4',
    type: 'human',
    content: "What's my name?",
    tool_calls: undefined
  },
  {
    id: 'chatcmpl-9zYV9mmpJrm3SQ7ngMJZ1XBHzHfL6',
    type: 'ai',
    content: 'Your name is Bob.',
    tool_calls: []
  }
]


## 以编程方式删除消息

我们还可以从图表内部以编程方式删除消息。在这里，我们将修改图表以在图表运行结束时删除所有旧消息（超过 3 条消息之前）。

In [6]:
import { RemoveMessage } from "@langchain/core/messages";
import { StateGraph, START, END } from "@langchain/langgraph";
import { MessagesAnnotation } from "@langchain/langgraph";

function deleteMessages(state: typeof MessagesAnnotation.State) {
  const messages = state.messages;
  if (messages.length > 3) {
    return { messages: messages.slice(0, -3).map(m => new RemoveMessage({ id: m.id })) };
  }
  return {};
}

// 我们需要修改逻辑以调用deleteMessages而不是立即结束
function shouldContinue2(state: typeof MessagesAnnotation.State): "action" | "delete_messages" {
  const lastMessage = state.messages[state.messages.length - 1];
  if (
    "tool_calls" in lastMessage &&
    Array.isArray(lastMessage.tool_calls) &&
    lastMessage.tool_calls.length
  ) {
    return "action";
  }
  // 否则，如果没有，我们就完成
  return "delete_messages";
}

// 定义一个新图
const workflow2 = new StateGraph(MessagesAnnotation)
  .addNode("agent", callModel)
  .addNode("action", toolNode)
  // 这是我们定义的新节点
  .addNode("delete_messages", deleteMessages)
  .addEdge(START, "agent")
  .addConditionalEdges(
    "agent",
    shouldContinue2
  )
  .addEdge("action", "agent")
  // 这是我们添加的新边缘：删除消息后，我们完成
  .addEdge("delete_messages", END);

const app2 = workflow2.compile({ checkpointer: memory });

我们现在可以尝试一下。我们可以调用该图两次，然后检查状态

In [7]:
import { HumanMessage } from "@langchain/core/messages";
import { v4 as uuidv4 } from "uuid";

const config2 = { configurable: { thread_id: "3" }, streamMode: "values" as const };

const inputMessage3 = new HumanMessage({
  id: uuidv4(),
  content: "hi! I'm bob",
});

console.log("--- FIRST ITERATION ---\n");
for await (const event of await app2.stream(
  { messages: [inputMessage3] },
  config2
)) {
  console.log(event.messages.map((message) => [message._getType(), message.content]));
}

const inputMessage4 = new HumanMessage({
  id: uuidv4(),
  content: "what's my name?",
});

console.log("\n\n--- SECOND ITERATION ---\n");
for await (const event of await app2.stream(
  { messages: [inputMessage4] },
  config2
)) {
  console.log(event.messages.map((message) => [message._getType(), message.content]), "\n");
}

--- FIRST ITERATION ---

[ [ 'human', "hi! I'm bob" ] ]


[
  [ 'human', "hi! I'm bob" ],
  [ 'ai', 'Hi Bob! How can I assist you today?' ]
]


--- SECOND ITERATION ---

[
  [ 'human', "hi! I'm bob" ],
  [ 'ai', 'Hi Bob! How can I assist you today?' ],
  [ 'human', "what's my name?" ]
] 

[
  [ 'human', "hi! I'm bob" ],
  [ 'ai', 'Hi Bob! How can I assist you today?' ],
  [ 'human', "what's my name?" ],
  [ 'ai', "Based on what you've told me, your name is Bob." ]
] 

[
  [ 'ai', 'Hi Bob! How can I assist you today?' ],
  [ 'human', "what's my name?" ],
  [ 'ai', "Based on what you've told me, your name is Bob." ]
] 



如果我们现在检查状态，我们应该看到它只有三个消息长。这是因为我们刚刚删除了之前的消息 - 否则将是四条！

In [8]:
const messages3 = (await app.getState(config2)).values["messages"]
console.dir(
  messages3.map((msg) => ({
    id: msg.id,
    type: msg._getType(),
    content: msg.content,
    tool_calls:
    msg.tool_calls,
  })),
  { depth: null }
);

[
  {
    id: 'chatcmpl-9zYVAEiiC9D7bb0wF4KLXgY0OAG8O',
    type: 'ai',
    content: 'Hi Bob! How can I assist you today?',
    tool_calls: []
  },
  {
    id: 'b93e5f35-cfa3-4ca6-9b59-154ce2bd476b',
    type: 'human',
    content: "what's my name?",
    tool_calls: undefined
  },
  {
    id: 'chatcmpl-9zYVBHJWtEM6pw2koE8dykzSA0XSO',
    type: 'ai',
    content: "Based on what you've told me, your name is Bob.",
    tool_calls: []
  }
]


请记住，删除消息时您需要确保剩余的消息列表仍然有效。该消息列表**实际上可能不是** - 这是因为它当前以 AI 消息开头，而某些模型不允许这样做。